In [1]:
# %pip install pydantic
# %pip install python-dotenv
# %pip install openai

In [4]:
import os
from openai import OpenAI
from pydantic import BaseModel, Field
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

class ProductInfo(BaseModel):
    product_description: list[str] = Field(description="The list of descriptions of the all products in the page")
    product_price: list[str] = Field(description="The list of prices of the all products in the page")
    product_rating: list[str] = Field(description="The list of ratings of the all products in the page")
    product_brand: list[str] = Field(description="The list of brands of the all products in the page")

def load_file(file_path: str) -> str:
    with open(file_path, "r") as file:
        return file.read()

def extract_product_info_with_chatgpt(product_page_raw_text: str) -> ProductInfo:
    response = client.beta.chat.completions.parse(
        model="gpt-4.1",
        messages=[
            {"role": "system", "content": """
             You are a helpful assistant that extracts product information from an amazon page text.
             You will be given a raw product page text and you will need to extract the product information:
             - product description
             - product price
             - product rating
             - product brand
             """},
            {"role": "user", "content": product_page_raw_text}
        ],
        temperature=0,
        response_format=ProductInfo
    )
    
    return response.choices[0].message.parsed

def transform_product_info_to_table(product_info: ProductInfo) -> pd.DataFrame:
    columns = ["product_description", "product_price", "product_rating", "product_brand"]
    # Zip the lists together to create rows for each product
    data = list(zip(
        product_info.product_description,
        product_info.product_price,
        product_info.product_rating,
        product_info.product_brand
    ))
    df = pd.DataFrame(data, columns=columns)
    return df

file_path = "/Users/greatmaster/Desktop/projects/oreilly-live-trainings/oreilly_live_training_llm_apps/notebooks/assets-resources/shoes_product_page_raw.txt"

product_page_text = load_file(file_path)

product_page_text

'Saltar al contenido principal\nEnviar a LucasLisboa 1300-032\u200c\nTodos los departamentos\n\nTodos los departamentos\nBuscar en Amazon Business\nbasketball shoes men\n ES  \n\nHola LucasCuenta para Lucas Nicolosi, Unipessoal LDA \n0\nTodo\nConsigue la app \nPrime Day\nVolver a comprar\nAhorro empresarial\nTarjetas regalo \nCompra al por mayor\nDescuentos por cantidad\nTu punto de ahorro\nSuscríbete y ahorra\nAñadir usuarios\nSuministros Informáticos \nProductos personalizados\nMarcas de Amazon\nAñade a tus compañeros\nListasPrueba Business Prime \nAmazon.es\nLos más vendidos\nOfertas\nAmazon Segunda mano\nLista de deseos\nTarjetas regalo\nAmazon Prime\nApps de Amazon\nVender en Amazon\nTrabajar en Amazon\n1-48 de más de 3.000 resultados para "basketball shoes men"\nOrdenar por:\nDestacados\nOrdenar por:Destacados\n\nARRIGO BELLO Zapatillas Hombre Zapatos Sneakers Vestir Casual Deportivas Caminar Correr Deportivas Gimnasio Moda cómodo Viajar Talla 41-46 (A Gris,Talla_41)\nARRIGO BELL

In [5]:
product_info = extract_product_info_with_chatgpt(product_page_text)
print(transform_product_info_to_table(product_info))

                                  product_description  \
0   Own The Game 3 Shoes, Zapatillas de Baloncesto...   
1                  Court Vision Lo Nn, Sneaker Hombre   
2   UA Lockdown 7, Zapatillas de Baloncesto Unisex...   
3   Zapatillas de Deporte Slip On Zapatillas Runni...   
4                Rebound V6, Zapatillas Unisex Adulto   
..                                                ...   
56  Zapatillas de Estar por Casa Zuecos Mujer Homb...   
57  Zuecos para Hombre y Mujer Ligeros Zapatillas ...   
58  Sandalias Hombre, Sandalias de Senderismo para...   
59  Zapatillas Hombre Zapatos Sneakers Vestir Casu...   
60  Zapatos de Seguridad para Hombre Mujer Ligeros...   

                                   product_price  \
0   53,68€ IVA no incluido / 66,03€ IVA incluido   
1   46,40€ IVA no incluido / 57,07€ IVA incluido   
2   45,45€ IVA no incluido / 55,90€ IVA incluido   
3                                         30,49€   
4   34,67€ IVA no incluido / 42,64€ IVA incluido   
.. 

In [2]:
import os
# loading from a .env file
# load_dotenv(dotenv_path="/full/path/to/your/.env")

# or 
# if you're on google colab just uncomment below and replace with your openai api key
# os.environ["OPENAI_API_KEY"] = "<your-openai-api-key>"

In [22]:
MODEL = 'gpt-4o-mini'

In [23]:
from openai import OpenAI
client = OpenAI()

def recommender_chat_bot(user_query):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "system", "content": "You are a chatbot system \
            and recommender for products about tech to a diverse group of users."},
                  {"role": "user", "content": user_query}]
    )
    
    return response.choices[0].message.content

In [24]:
def generate_data_with_chatgpt(data_example):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "system", "content": "You generate examples of tech products."},
                  {"role": "user", "content": data_example}]
    )
    
    return response.choices[0].message.content


tech_products_prompt = "Create 20 examples of tech products\
    with their descriptions as a list of bullet points"

data_tech = generate_data_with_chatgpt(tech_products_prompt)
data_tech

'Here are 20 examples of tech products, along with their descriptions:\n\n- **Smartphone XZ Pro**\n  - A flagship smartphone with a 6.8-inch OLED display, triple-lens camera with 108MP main sensor, and 5G connectivity.\n\n- **Wireless Noise-Cancelling Headphones**\n  - Over-ear headphones with adaptive noise cancellation, 30-hour battery life, and touch controls for seamless audio management.\n\n- **4K Ultra HD Smart TV**\n  - A 55-inch smart television with HDR support, voice-activated remote, and access to popular streaming services like Netflix and Hulu.\n\n- **Smart Home Hub**\n  - A central device that connects and controls smart home devices, including lights, thermostats, and security systems through voice commands and a dedicated app.\n\n- **Fitness Tracker Pro**\n  - Wearable device that monitors heart rate, sleep patterns, and activity levels with built-in GPS and smartphone notifications.\n\n- **VR Gaming Headset**\n  - Immersive virtual reality headset with 120Hz refresh ra

In [25]:
from IPython.display import Markdown
Markdown(data_tech)

Here are 20 examples of tech products, along with their descriptions:

- **Smartphone XZ Pro**
  - A flagship smartphone with a 6.8-inch OLED display, triple-lens camera with 108MP main sensor, and 5G connectivity.

- **Wireless Noise-Cancelling Headphones**
  - Over-ear headphones with adaptive noise cancellation, 30-hour battery life, and touch controls for seamless audio management.

- **4K Ultra HD Smart TV**
  - A 55-inch smart television with HDR support, voice-activated remote, and access to popular streaming services like Netflix and Hulu.

- **Smart Home Hub**
  - A central device that connects and controls smart home devices, including lights, thermostats, and security systems through voice commands and a dedicated app.

- **Fitness Tracker Pro**
  - Wearable device that monitors heart rate, sleep patterns, and activity levels with built-in GPS and smartphone notifications.

- **VR Gaming Headset**
  - Immersive virtual reality headset with 120Hz refresh rate, wide field of view, and motion tracking for an unparalleled gaming experience.

- **Portable Solar Charger**
  - Eco-friendly charging solution for mobile devices, featuring high-efficiency solar panels and dual USB ports for simultaneous charging.

- **E-reader 3000**
  - Lightweight tablet designed for reading with glare-free display technology, adjustable front light, and weeks-long battery life.

- **Smart Refrigerator**
  - Energy-efficient fridge with a touchscreen interface, internal cameras for inventory tracking, and compatibility with grocery delivery apps.

- **Robotic Vacuum Cleaner**
  - Autonomous vacuum with smart mapping technology, scheduling capabilities, and voice control integration with Alexa and Google Assistant.

- **Wireless Charging Pad**
  - Sleek charging base that supports fast wireless charging for compatible smartphones and earbuds with a non-slip design.

- **Gaming Laptop Elite**
  - High-performance laptop equipped with the latest graphics card, 16GB RAM, and RGB backlit keyboard, optimized for gaming.

- **Smartwatch 2.0**
  - Stylish smartwatch with health tracking features, customizable watch faces, and seamless integration with smartphones for notifications.

- **Drone Camera Pro**
  - Compact drone featuring a 4K camera, 3-axis gimbal stabilization, and intelligent flight modes for aerial photography enthusiasts.

- **Home Security Camera**
  - Wi-Fi-enabled indoor/outdoor security camera with night vision, motion detection alerts, and two-way audio for real-time communication.

- **Digital Whiteboard**
  - Interactive digital display for workplaces or classrooms, allowing for collaborative brainstorming, screen sharing, and notes capturing.

- **Bluetooth Smart Lock**
  - Keyless entry lock that can be managed through a smartphone app, offering remote access and temporary codes for visitors.

- **Wearable Medical Monitor**
  - Device that continuously monitors vital signs such as heart rate and oxygen levels, designed for both fitness tracking and chronic condition management.

- **Streaming Media Player**
  - Compact device that plugs into your TV and streams content from all major platforms with voice search functionality.

- **AI-Powered Language Translator**
  - Portable device that translates spoken language in real-time, supporting multiple languages and featuring a long-lasting battery for travel convenience.

- **Advanced Gaming Console**
  - Next-gen gaming system capable of 8K gaming, with cloud gaming support and an extensive library of exclusive titles. 

These products span various categories, including consumer electronics, smart home devices, and wearables.

In [28]:
from pydantic import BaseModel, Field
from typing import List

class ProductInfoStructured(BaseModel):
    name: str = Field(..., title="Product Name", description="The name of the product")
    description: str = Field(..., title="Product Description", description="The description of the product")

class ProductInfoStructuredTable(BaseModel):
    products: List[ProductInfoStructured] = Field(..., title="Products", description="A list of products with their names and descriptions")


def create_structured_table(products_info_raw):
    response = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You create a structured table of all the products in the input \
            with their names and descriptions. The table will have 2 columns: name and description."},
                  {"role": "user", "content": products_info_raw}],
        response_format=ProductInfoStructuredTable,
        temperature=0.0
    )
    
    return response.choices[0].message.parsed


structured_table = create_structured_table(data_tech)
structured_table

ProductInfoStructuredTable(products=[ProductInfoStructured(name='Smartphone XZ Pro', description='A flagship smartphone with a 6.8-inch OLED display, triple-lens camera with 108MP main sensor, and 5G connectivity.'), ProductInfoStructured(name='Wireless Noise-Cancelling Headphones', description='Over-ear headphones with adaptive noise cancellation, 30-hour battery life, and touch controls for seamless audio management.'), ProductInfoStructured(name='4K Ultra HD Smart TV', description='A 55-inch smart television with HDR support, voice-activated remote, and access to popular streaming services like Netflix and Hulu.'), ProductInfoStructured(name='Smart Home Hub', description='A central device that connects and controls smart home devices, including lights, thermostats, and security systems through voice commands and a dedicated app.'), ProductInfoStructured(name='Fitness Tracker Pro', description='Wearable device that monitors heart rate, sleep patterns, and activity levels with built-i

In [29]:
structured_table.products

[ProductInfoStructured(name='Smartphone XZ Pro', description='A flagship smartphone with a 6.8-inch OLED display, triple-lens camera with 108MP main sensor, and 5G connectivity.'),
 ProductInfoStructured(name='Wireless Noise-Cancelling Headphones', description='Over-ear headphones with adaptive noise cancellation, 30-hour battery life, and touch controls for seamless audio management.'),
 ProductInfoStructured(name='4K Ultra HD Smart TV', description='A 55-inch smart television with HDR support, voice-activated remote, and access to popular streaming services like Netflix and Hulu.'),
 ProductInfoStructured(name='Smart Home Hub', description='A central device that connects and controls smart home devices, including lights, thermostats, and security systems through voice commands and a dedicated app.'),
 ProductInfoStructured(name='Fitness Tracker Pro', description='Wearable device that monitors heart rate, sleep patterns, and activity levels with built-in GPS and smartphone notificatio

In [30]:
import pandas as pd

# Convert list of ProductInfoStructured objects to dataframe
products_df = pd.DataFrame([
    {"name": product.name, "description": product.description} 
    for product in structured_table.products
])

products_df

,name,description
0,Smartphone XZ Pro,A flagship smartphone with a 6.8-inch OLED dis...
1,Wireless Noise-Cancelling Headphones,Over-ear headphones with adaptive noise cancel...
2,4K Ultra HD Smart TV,"A 55-inch smart television with HDR support, v..."
3,Smart Home Hub,A central device that connects and controls sm...
4,Fitness Tracker Pro,"Wearable device that monitors heart rate, slee..."
5,VR Gaming Headset,Immersive virtual reality headset with 120Hz r...
6,Portable Solar Charger,Eco-friendly charging solution for mobile devi...
7,E-reader 3000,Lightweight tablet designed for reading with g...
8,Smart Refrigerator,Energy-efficient fridge with a touchscreen int...
9,Robotic Vacuum Cleaner,Autonomous vacuum with smart mapping technolog...


In [27]:
structured_table.name
structured_table.description

'A flagship smartphone with a 6.8-inch OLED display, triple-lens camera with 108MP main sensor, and 5G connectivity.'

In [31]:
def create_costumer_profile(user_query):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "system", "content": "You create a list of customer \
            profiles based on input."},
                  {"role": "user", "content": user_query}]
    )
    
    return response.choices[0].message.content


create_costumer_profile = create_costumer_profile("Create 10 customer profiles \
    for users with different backgrounds and preferences.")

Markdown(create_costumer_profile)

Sure! Here are 10 diverse customer profiles based on various backgrounds, preferences, and lifestyles:

### Customer Profile 1: Tech-Savvy Millennial
- **Name:** Sarah Thompson
- **Age:** 29
- **Occupation:** Software Developer
- **Location:** San Francisco, CA
- **Interests:** Coding, Video Games, Sustainable Tech
- **Preferences:** Prefers eco-friendly gadgets and tech services, values online reviews, and enjoys gamified learning platforms.

### Customer Profile 2: Health-Conscious Retiree
- **Name:** John Peterson
- **Age:** 67
- **Occupation:** Retired Teacher
- **Location:** Sarasota, FL
- **Interests:** Gardening, Walking, Healthy Cooking
- **Preferences:** Looks for organic and local produce, prefers low-impact exercise classes, enjoys cooking classes and health blogs.

### Customer Profile 3: Fashion-Forward College Student
- **Name:** Emily Chen
- **Age:** 21
- **Occupation:** Fashion Design Student
- **Location:** New York City, NY
- **Interests:** Fashion Trends, Social Media, DIY Projects
- **Preferences:** Shops at thrift stores, influenced by social media trends, enjoys brands that support sustainable fashion.

### Customer Profile 4: Busy Professional Parent
- **Name:** Michael Rodriguez
- **Age:** 38
- **Occupation:** Marketing Manager
- **Location:** Chicago, IL
- **Interests:** Family Activities, Outdoors, Cooking
- **Preferences:** Values convenience (meal kits, online grocery shopping), looks for family-friendly entertainment, often seeks work-life balance tips.

### Customer Profile 5: Eco-Conscious Urbanite
- **Name:** Leah Patel
- **Age:** 34
- **Occupation:** Environmental Scientist
- **Location:** Portland, OR
- **Interests:** Outdoor Activities, Sustainability, Volunteering
- **Preferences:** Supports local businesses, primarily buys recycled or upcycled products, prefers brands that align with environmental values.

### Customer Profile 6: Adventurous Traveler
- **Name:** James Brown
- **Age:** 45
- **Occupation:** Travel Blogger
- **Location:** Austin, TX
- **Interests:** Hiking, Cultural Experiences, Photography
- **Preferences:** Looks for unique travel experiences, enjoys off-the-beaten-path accommodations, values authentic cultural interactions.

### Customer Profile 7: Budget-Conscious Shopper
- **Name:** Amanda Wilkes
- **Age:** 50
- **Occupation:** Administrative Assistant
- **Location:** Atlanta, GA
- **Interests:** DIY Projects, Cooking, Couponing
- **Preferences:** Shop sales and use coupons, values affordability over brand loyalty, enjoys thrift shopping for unique finds.

### Customer Profile 8: Luxury Seeker
- **Name:** Frederick King
- **Age:** 55
- **Occupation:** CEO of a Financial Firm
- **Location:** Miami, FL
- **Interests:** Fine Dining, Golf, Luxury Travel
- **Preferences:** Prefers high-end brands, values exclusivity and personalized service, enjoys premium experiences and luxury goods.

### Customer Profile 9: Tech Blogging Gen Z
- **Name:** Olivia Green
- **Age:** 22
- **Occupation:** Content Creator
- **Location:** Los Angeles, CA
- **Interests:** Social Media, Technology, Gaming
- **Preferences:** Engages with interactive brands on social media, prefers tech products that enhance online presence, seeks after-sale support for gadgets.

### Customer Profile 10: Community-Oriented Non-Profit Worker
- **Name:** David Kim
- **Age:** 36
- **Occupation:** Non-Profit Coordinator
- **Location:** Seattle, WA
- **Interests:** Social Causes, Community Events, Art
- **Preferences:** Supports brands that give back to the community, attends local events and festivals, enjoys products that promote social good.

These profiles can help tailor marketing strategies or product offerings to better meet the needs and preferences of varied customer segments.

In [34]:
from pydantic import BaseModel
from typing import List

class CustomerProfileStructured(BaseModel):
    name: str
    age: int 
    occupation: str
    location: str
    interests: str
    buying_preferences: str

class CustomerProfileStructuredTable(BaseModel):
    profiles: List[CustomerProfileStructured]

def create_structured_profile_table(profile_data: str):
    response = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Extract customer profile information into structured data with name, age, occupation, location, interests, and buying preferences fields"},
            {"role": "user", "content": f"Extract the profile information from this text into structured data: {profile_data}"}
        ],
        response_format=CustomerProfileStructuredTable,
        temperature=0.0
    )
    
    # Parse the JSON response into our Pydantic models
    profile_data = response.choices[0].message.parsed
    
    return profile_data

structured_profiles = create_structured_profile_table(create_costumer_profile)
structured_profiles


CustomerProfileStructuredTable(profiles=[CustomerProfileStructured(name='Sarah Thompson', age=29, occupation='Software Developer', location='San Francisco, CA', interests='Coding, Video Games, Sustainable Tech', buying_preferences='Prefers eco-friendly gadgets and tech services, values online reviews, and enjoys gamified learning platforms.'), CustomerProfileStructured(name='John Peterson', age=67, occupation='Retired Teacher', location='Sarasota, FL', interests='Gardening, Walking, Healthy Cooking', buying_preferences='Looks for organic and local produce, prefers low-impact exercise classes, enjoys cooking classes and health blogs.'), CustomerProfileStructured(name='Emily Chen', age=21, occupation='Fashion Design Student', location='New York City, NY', interests='Fashion Trends, Social Media, DIY Projects', buying_preferences='Shops at thrift stores, influenced by social media trends, enjoys brands that support sustainable fashion.'), CustomerProfileStructured(name='Michael Rodriguez'

In [35]:
# Convert structured profiles to pandas DataFrame for easier analysis
import pandas as pd

profiles_df = pd.DataFrame([
    {
        "name": profile.name,
        "age": profile.age,
        "occupation": profile.occupation,
        "location": profile.location, 
        "interests": profile.interests,
        "buying_preferences": profile.buying_preferences
    }
    for profile in structured_profiles.profiles
])

profiles_df

,name,age,occupation,location,interests,buying_preferences
0,Sarah Thompson,29,Software Developer,"San Francisco, CA","Coding, Video Games, Sustainable Tech",Prefers eco-friendly gadgets and tech services...
1,John Peterson,67,Retired Teacher,"Sarasota, FL","Gardening, Walking, Healthy Cooking","Looks for organic and local produce, prefers l..."
2,Emily Chen,21,Fashion Design Student,"New York City, NY","Fashion Trends, Social Media, DIY Projects","Shops at thrift stores, influenced by social m..."
3,Michael Rodriguez,38,Marketing Manager,"Chicago, IL","Family Activities, Outdoors, Cooking","Values convenience (meal kits, online grocery ..."
4,Leah Patel,34,Environmental Scientist,"Portland, OR","Outdoor Activities, Sustainability, Volunteering","Supports local businesses, primarily buys recy..."
5,James Brown,45,Travel Blogger,"Austin, TX","Hiking, Cultural Experiences, Photography","Looks for unique travel experiences, enjoys of..."
6,Amanda Wilkes,50,Administrative Assistant,"Atlanta, GA","DIY Projects, Cooking, Couponing","Shop sales and use coupons, values affordabili..."
7,Frederick King,55,CEO of a Financial Firm,"Miami, FL","Fine Dining, Golf, Luxury Travel","Prefers high-end brands, values exclusivity an..."
8,Olivia Green,22,Content Creator,"Los Angeles, CA","Social Media, Technology, Gaming",Engages with interactive brands on social medi...
9,David Kim,36,Non-Profit Coordinator,"Seattle, WA","Social Causes, Community Events, Art",Supports brands that give back to the communit...


In [36]:
user_info = profiles_df.iloc[9]


def recommend_product(profile):
    prompt_to_recommender = f"Recommend a tech product from this list ONLY:\n\n{data_tech} \
        for this customer profile: {profile}.\
        Output ONLY A SIMPLE MESSAGE REFERENCING THE CUSTOMER AND THE recommendations."
    
    recommendation = recommender_chat_bot(prompt_to_recommender)
    
    return recommendation


product_recommendation = recommend_product(user_info)

print(user_info)
Markdown(product_recommendation)

name                                                          David Kim
age                                                                  36
occupation                                       Non-Profit Coordinator
location                                                    Seattle, WA
interests                          Social Causes, Community Events, Art
buying_preferences    Supports brands that give back to the communit...
Name: 9, dtype: object


Hi David Kim, based on your interests in social causes and support for community-focused brands, I recommend the **Portable Solar Charger**. It's an eco-friendly option that not only helps you stay connected but also aligns with your values of sustainability and giving back to the environment.

In [16]:
data_tech

"Sure! Here are 20 examples of tech products along with their descriptions:\n\n1. **Smartphone Charging Dock**\n   - A sleek, wireless charging dock that can charge multiple devices simultaneously with fast-charging capabilities and LED indicators.\n\n2. **Fitness Tracker**\n   - A waterproof fitness tracker that monitors heart rate, sleep patterns, and activity levels, syncing data with a mobile app for personalized health insights.\n\n3. **Wireless Noise-Canceling Headphones**\n   - Over-ear headphones featuring state-of-the-art noise-cancelation technology, extended battery life, and touch controls for enhanced audio experiences.\n\n4. **Smart Home Hub**\n   - A central device that connects various smart home products, allowing users to manage lighting, security, and temperature through a single app or voice command.\n\n5. **Portable Solar Charger**\n   - An eco-friendly solar charger designed for outdoor enthusiasts, featuring multiple USB ports and a rugged, waterproof design for 

In [17]:
from pydantic import BaseModel, Field
from typing import List


class TechProduct(BaseModel):
    names: List[str] = Field(..., title="All the product names", description="The titles of the products in question")
    descriptions: List[str] = Field(..., title="All the product descriptions", description="A complete list of descriptions of the products extracted from the source.")
    
def extract_tech_product_data(raw_tech_data: str):
    response = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {'role': 'system', 'content': 'You are an extraction engine for tech products, \
                you extract the following fields:\
                1. name - list of all names of products in the input\
                2. description - list of all descritionps in the products\
                    Make sure to extract all the names and descriptions from the input.'},
        ],
        response_format=TechProduct,
        temperature=0.0
    )
    
    return response.choices[0].message

tech_data_structured = extract_tech_product_data(f'Extract all the names and descriptions from this raw data: \n\n {data_tech}')
tech_data_structured

ParsedChatCompletionMessage[TechProduct](content='{"names":["iPhone 15","Samsung Galaxy S23","Google Pixel 8","MacBook Pro 2023","Dell XPS 13"],"descriptions":["The latest iPhone model featuring a new A17 Bionic chip, improved camera system, and longer battery life.","Samsung\'s flagship smartphone with a dynamic AMOLED display, advanced camera features, and 5G connectivity.","Google\'s newest Pixel phone offering an upgraded Tensor chip, enhanced AI capabilities, and a stunning OLED display.","Apple\'s high-performance laptop with the M3 chip, Retina display, and all-day battery life.","Dell\'s ultra-portable laptop with a 13-inch InfinityEdge display, 11th Gen Intel processors, and a sleek design."]}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[], parsed=TechProduct(names=['iPhone 15', 'Samsung Galaxy S23', 'Google Pixel 8', 'MacBook Pro 2023', 'Dell XPS 13'], descriptions=['The latest iPhone model featuring a new A17 Bionic chip, improved camera syst

In [18]:
tech_data_structured.parsed.names

['iPhone 15',
 'Samsung Galaxy S23',
 'Google Pixel 8',
 'MacBook Pro 2023',
 'Dell XPS 13']

In [19]:
tech_data_structured.parsed.descriptions

['The latest iPhone model featuring a new A17 Bionic chip, improved camera system, and longer battery life.',
 "Samsung's flagship smartphone with a dynamic AMOLED display, advanced camera features, and 5G connectivity.",
 "Google's newest Pixel phone offering an upgraded Tensor chip, enhanced AI capabilities, and a stunning OLED display.",
 "Apple's high-performance laptop with the M3 chip, Retina display, and all-day battery life.",
 "Dell's ultra-portable laptop with a 13-inch InfinityEdge display, 11th Gen Intel processors, and a sleek design."]

In [20]:
def create_diagrams_with_AI(user_query):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "system", "content": "You create mermaid diagrams with AI."},
                  {"role": "user", "content": user_query}]
    )
    
    return response.choices[0].message.content

output_diagram_raw = create_diagrams_with_AI("Create a diagram of a flowchart that shows the process of a customer \
    buying a product from an e-commerce website.")

Markdown(output_diagram_raw)

Here’s a flowchart diagram created in Mermaid syntax that outlines the process of a customer buying a product from an e-commerce website:

```mermaid
flowchart TD
    A[Start] --> B[Customer visits e-commerce website]
    B --> C[Customer browses products]
    C --> D{Selects a product?}
    D -->|Yes| E[View product details]
    D -->|No| C
    E --> F[Add product to cart]
    F --> G{Continue shopping?}
    G -->|Yes| C
    G -->|No| H[Go to checkout]
    H --> I[Enter shipping information]
    I --> J[Choose payment method]
    J --> K[Review order]
    K --> L{Confirm order?}
    L -->|Yes| M[Process payment]
    M --> N[Order confirmation]
    L -->|No| K
    N --> O[End]
```

You can visualize this flowchart using a Mermaid renderer, which will clearly show the steps involved in the purchasing process on an e-commerce website.

In [21]:
class MermaidDiagram(BaseModel):
    diagram: str = Field(..., title="Mermaid Diagram", description="The mermaid diagram as a properly formatted markdown ready to visualize.")

def create_diagrams_with_AI(user_query):
    response = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[{"role": "system", "content": "You create mermaid diagrams with AI ready to be visualized in markdown."},
                  {"role": "user", "content": user_query}],
        response_format=MermaidDiagram,
        temperature=0.0
    )
    
    return response.choices[0].message.parsed.diagram

output_diagram_raw = create_diagrams_with_AI("Create a diagram of a flowchart that shows the process of a customer \
    buying a product from an e-commerce website.")

Markdown(output_diagram_raw)

```mermaid
flowchart TD
    A[Start] --> B[Visit E-commerce Website]
    B --> C[Browse Products]
    C --> D[Select Product]
    D --> E[Add to Cart]
    E --> F[View Cart]
    F --> G[Proceed to Checkout]
    G --> H[Enter Shipping Information]
    H --> I[Enter Payment Information]
    I --> J[Review Order]
    J --> K[Place Order]
    K --> L[Receive Confirmation]
    L --> M[End]
```